In [1]:
import numpy as np
import pandas as pd
import json

# Regression
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import PolynomialFeatures
from RecursiveSymbolicRegression import RecursiveSymbolicRegression

# Support
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline

/home/olivetti/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/olivetti/anaconda3/lib/python3.6/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
# Carrega info dos data sets
# Aplica algoritmo e retorna: mae, msre, # termos, ground truth termos
# Aplica outros algoritmos de regressão e monta duas tabelas: mae e msre

# algoritmos: linear regression, kernel regression, xgboost, polynomial features

# Kernel Ridge e Linear Regr. # termos = # de vars
# Poly e xgboost pode medir

# Table 1:
'''
Algorithm | MAE | MSE | # terms | dataname | function
===============================
Ground Truth | 0 | 0 | n | name
===============================
LR | ... | vars
KR | ... | vars
xgboost | ... | n*m
poly | ... | n^p
===============================
SR | ... | k
===============================
GP | .... | --
===============================
'''

'\nAlgorithm | MAE | MSE | # terms | dataname | function\n===============================\nGround Truth | 0 | 0 | n | name\n===============================\nLR | ... | vars\nKR | ... | vars\nxgboost | ... | n*m\npoly | ... | n^p\n===============================\nSR | ... | k\n===============================\nGP | .... | --\n===============================\n'

In [3]:
# Load data sets info
# info = {'dataset': {'functions': {'name':n_terms}, 'has_test': {True|False}}}
with open('Benchmark/data.info') as f:
    info = json.loads(f.read())
f.closed
info

{'Bee': {'functions': {'F1': 3,
   'F10': 1,
   'F12': 3,
   'F13': 1,
   'F14': 1,
   'F15': 3,
   'F16': 1,
   'F2': 4,
   'F3': 5,
   'F4': 6,
   'F5': 2,
   'F6': 2,
   'F7': 2,
   'F8': 1,
   'F9': 2},
  'has_test': False},
 'Neat': {'functions': {'F1': 4,
   'F10': 2,
   'F2': 5,
   'F3': 2,
   'F4': 2,
   'F5': 1,
   'F6': 1,
   'F7': 1,
   'F8': 2,
   'F9': 1},
  'has_test': False},
 'Surrogate1': {'functions': {'F1': 1, 'F2': 1, 'F3': 1, 'F4': 1, 'F5': 3},
  'has_test': True},
 'Surrogate2': {'functions': {'F1': 3,
   'F2': 1,
   'F3': 1,
   'F4': 1,
   'F5': 1,
   'F6': 3,
   'F7': 1},
  'has_test': True}}

In [4]:
PolynomialRegression = Pipeline([('pf', PolynomialFeatures()),('lr', LinearRegression())])

regressors = {'LR': LinearRegression(),
              'GB': GradientBoostingRegressor(), 'PF': PolynomialRegression}

parameters_set = { 'LR': {}, 
                  'SVR': {'kernel': ['rbf', 'poly'], 'gamma': np.arange(0.1,1,0.1), 'degree':[2,3,4,5]},
                  'GB': {'max_depth': list(range(3,6)), 'n_estimators': [100,500]},
                  'PF': {'pf__degree': list(range(2,6))}
                  }

In [5]:
def loadData( dataset, function, has_test ):
    Z_test = None
    if has_test:
        Z_train = np.loadtxt('Benchmark/{}/{}_train.csv'.format(dataset, function))
        Z_test = np.loadtxt('Benchmark/{}/{}_test.csv'.format(dataset, function))
    else:
        Z_train = np.loadtxt('Benchmark/{}/{}.csv'.format(dataset, function))
        
    return Z_train, Z_test

def get_terms(clf, name):
    clf = clf.best_estimator_
    if name == 'LR':
        return len(clf.coef_) 
    elif name == 'GB':
        return len(clf.feature_importances_) * len(clf.estimators_)
    elif name == 'PF':
        return clf.named_steps['pf'].n_output_features_     

def pSqRoot(x):
    return np.sqrt(np.abs(x))

def runExperiments( dataset, function, has_test_val ):
    table = []
    
    Z_train, Z_test = loadData( dataset, function, has_test_val )
    
    X_train, y_train = Z_train[:,:-1], Z_train[:,-1]
    if Z_test is None:
        X_test, y_test = X_train.copy(), y_train.copy()
    else:
        X_test, y_test = Z_test[:,:-1], Z_test[:,-1]
    
    for name, regressor in regressors.items():
        clf = GridSearchCV(regressor, parameters_set[name], cv=5)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        mae = np.abs(y_pred - y_test).mean()
        mse = np.square(y_pred - y_test).mean()
        nterms = get_terms(clf, name)
        table.append([name, mae, mse, nterms, dataset, function])
        
    sr = RecursiveSymbolicRegression(
        LinearModel=LinearRegression, 
        functions=[np.sin, np.cos, np.tan, pSqRoot, np.log1p, np.log]
    )
    sr.fit(X_train, y_train)
    y_pred = sr.predict(X_test)
    mae = np.abs(y_pred - y_test).mean()
    mse = np.square(y_pred - y_test).mean()   
        
    nterms = sr.terms 
    table.append(['RSR', mae, mse, nterms, dataset, function])
    #if nterms < 20:
    #    print(sr.printExpr(sr.fitexpr, sr.fitcoef, sr.fitbias))
    
    return table, sr.printExpr(sr.fitexpr, sr.fitcoef, sr.fitbias)

In [6]:
results = []
expression = {}

for dataset in [ 'Bee']:#,'Neat', 'Surrogate1', 'Surrogate2']: 
    print(dataset)
    expression[dataset] = {}
    for function in sorted(info[dataset]['functions']):
        table, expr = runExperiments(dataset, function, info[dataset]['has_test'])
        table.append(['OPT', 0.0, 0.0, info[dataset]['functions'][function], dataset, function])
        results.append(table)
        expression[dataset][function] = expr
        for alg, mae, mse, nterms, name, func in table:
            print('{} | {:.2f} | {:.2f} | {} | {} | {}'.format(alg, mae, mse, nterms, name, func))
        print('===============================')


Bee


/home/olivetti/Projects/MonteCarloSymbolicRegression/RecursiveSymbolicRegression.py:122: RuntimeWarning: invalid value encountered in log
  y = f(x)


LR | 0.19 | 0.06 | 1 | Bee | F1
GB | 0.00 | 0.00 | 100 | Bee | F1
PF | 0.00 | 0.00 | 4 | Bee | F1
RSR | 0.00 | 0.00 | 3 | Bee | F1
OPT | 0.00 | 0.00 | 3 | Bee | F1


/home/olivetti/Projects/MonteCarloSymbolicRegression/RecursiveSymbolicRegression.py:122: RuntimeWarning: invalid value encountered in log
  y = f(x)


LR | 0.12 | 0.03 | 2 | Bee | F10
GB | 0.00 | 0.00 | 1000 | Bee | F10
PF | 0.00 | 0.00 | 21 | Bee | F10
RSR | 0.11 | 0.02 | 4 | Bee | F10
OPT | 0.00 | 0.00 | 1 | Bee | F10


/home/olivetti/Projects/MonteCarloSymbolicRegression/RecursiveSymbolicRegression.py:122: RuntimeWarning: invalid value encountered in log
  y = f(x)


LR | 0.22 | 0.07 | 1 | Bee | F12
GB | 0.00 | 0.00 | 100 | Bee | F12
PF | 0.00 | 0.00 | 6 | Bee | F12
RSR | 0.00 | 0.00 | 2 | Bee | F12
OPT | 0.00 | 0.00 | 3 | Bee | F12


/home/olivetti/Projects/MonteCarloSymbolicRegression/RecursiveSymbolicRegression.py:122: RuntimeWarning: invalid value encountered in log
  y = f(x)


LR | 0.04 | 0.00 | 1 | Bee | F13
GB | 0.00 | 0.00 | 100 | Bee | F13
PF | 0.00 | 0.00 | 6 | Bee | F13
RSR | 0.02 | 0.00 | 3 | Bee | F13
OPT | 0.00 | 0.00 | 1 | Bee | F13


KeyboardInterrupt: 

In [7]:
dataset = 'Bee'
function = 'F9'
table, sr = runExperiments(dataset, function, info[dataset]['has_test'])
for alg, mae, mse, nterms, name, func in table:
    print('{} | {:.6f} | {:.6f} | {} | {} | {}'.format(alg, mae, mse, nterms, name, func))
print('===============================')
print(sr)

/home/olivetti/Projects/MonteCarloSymbolicRegression/RecursiveSymbolicRegression.py:122: RuntimeWarning: invalid value encountered in log
  y = f(x)


LR | 0.224559 | 0.069402 | 2 | Bee | F9
GB | 0.004292 | 0.000033 | 200 | Bee | F9
PF | 0.001879 | 0.000005 | 21 | Bee | F9
RSR | 0.000000 | 0.000000 | 2 | Bee | F9
1.0*sin(x0) + 1.0*sin(x1**2) + 0.0


In [17]:
resultsFlat = [row for result in results for row in result]
algs, mae, mse, terms, dataname, function = zip(*resultsFlat)
df = pd.DataFrame({'algorithm':algs, 'MAE': mae, 'MSE':mse, 'Terms':terms, 'Dataset': dataname, 'Function': function})
df.to_csv('results.csv')

In [19]:
datanames = []
functions = []
exprs = []

for dataname in expression:
    for function in expression[dataname]:
        datanames.append(dataname)
        functions.append(function)
        exprs.append(expression[dataname][function])
        
df2 = pd.DataFrame({'Dataset':datanames, 'Function':functions, 'Expression':exprs})        
df2.to_csv('expressions.csv')

In [7]:
from itertools import product

dims = (1, 2, 3, 10)
orders = (1,2,3,4)
basis = (1,2,3,4)

#results2 = {}
#for dim in dims:
#    results2[dim] = {}
#    for base in basis:
#        results2[dim][base] = {}

for dim, order, base in [(10,4,4)]:#product(dims, orders, basis):
    if base > order:
        continue
    correct = 0
    for it in range(30):
        f, expr = build_function(dim, order, base)
        X = np.random.uniform(0,1,(4000,dim))
        y = f(X)
        sr = RecursiveSymbolicRegression(LinearModel=LinearRegression, functions=[])
        sr.fit(X[:2500,:], y[:2500], 3,0,0)
        y_pred = sr.predict(X[2500:,:])
        mae = np.abs(y_pred - y[2500:]).mean()
        nterms = sr.terms
        
        if sr.terms == base and mae < 1e-6:
            correct = correct + 1
        else:
            print(expr)
            if len(sr.fitcoef) < 8:
                print(sr.printExpr(sr.fitexpr, sr.fitcoef, sr.fitbias), mae)
        print(correct)
    print(dim, order, base, correct)
    results2[dim][order][base] = correct

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
0.06200589681564894*(X[:,0]**1)*(X[:,2]**1)*(X[:,3]**1)*(X[:,4]**1)+0.5641521032623424*(X[:,0]**1)*(X[:,1]**2)*(X[:,8]**1)+-0.03216730970341519*(X[:,0]**2)*(X[:,7]**1)*(X[:,9]**1)+-0.9463234025697946*(X[:,2]**1)*(X[:,4]**1)*(X[:,6]**1)*(X[:,9]**1)+0.4773427124712717
19
20
21
22
23
24
25
26
27
28
29
10 4 4 29


NameError: name 'results2' is not defined

In [6]:
def build_function(dim, order, base):
    
    # idx -> [(idx, exp) for idx in {0..dim-1}]
    base_f = {}
    for d in range(dim):
        base_f[d] = []
        for o in range(order+1):
            base_f[d].append((d,o))

    # basis = [ [(0,exp),...,(dim-1,exp)] for product() ]
    basis = [ list(p) for p in product(*base_f.values()) ]
    basis = [ b for b in basis if sum([e for i,e in b]) <= order ]
    basis = [ tuple([(i,e) for i,e in b if e!=0]) for b in basis ]
    basis = [ b for b in basis if len(b) ]
    basis = list(set(basis))
    
    basisO = [b for b in basis if sum([e for i,e in b])==order]
    
    expr = [list(basisO[np.random.choice(len(basisO))])]
    basis = [b for b in basis if b != tuple(expr[0])]
    if len(basis) > 0:
        expr = expr + [list(basis[idx]) for idx in np.random.choice(len(basis), base-1, replace=False)]
    
    expr = [(np.random.uniform(-1,1), term) for term in expr]
    finalE =   '+'.join([
                 '{}*'.format(c) +
                '*'.join(['(X[:,{}]**{})'.format(term, exp)  for term, exp in e])
                for c, e in expr
            ]) + '+{}'.format(np.random.uniform(-1,1))
    def f(X):
        return eval(finalE, {'X':X} )
    return f, finalE

    
#f, expr = build_function(3, 3, 2)
#X = np.random.uniform(0,1,(25,3))
#f(X)

In [59]:
dimens = []
orders = []
bases = []
values = []

for dim in results2:
    for order in results2[dim]:
        for base in results2[dim][order]:
            dimens.append(dim)
            orders.append(order)
            bases.append(base)
            values.append(results2[dim][order][base])

df3 = pd.DataFrame({'Dim': dimens, 'Order':orders, 'Base': bases, 'Count': values})            
df3.to_csv('result_poly_count.csv')

In [60]:
df3

Base  Count  Dim  Order
0      1     30    1      1
1      1     30    1      2
2      2     15    1      2
3      1     30    1      3
4      2     21    1      3
5      3     10    1      3
6      1     30    1      4
7      2     23    1      4
8      3     11    1      4
9      4      3    1      4
10     1      2    2      1
11     1     30    2      2
12     2     25    2      2
13     1     30    2      3
14     2     25    2      3
15     3     14    2      3
16     1     30    2      4
17     2     27    2      4
18     3     21    2      4
19     4     23    2      4
20     1      9    3      1
21     1     30    3      2
22     2     26    3      2
23     1     30    3      3
24     2     29    3      3
25     3     27    3      3
26     1     30    3      4
27     2     28    3      4
28     3     26    3      4
29     4     25    3      4
30     1      9   10      1
31     1     30   10      2
32     2     30   10      2
33     1     30   10      3
34     2     30   10      3
35     3     30   10      3
36     1     30   10      4
37     2     30   10      4
38     3     30   10      4
39     4     30   10      4

In [21]:
set([(1,2),(3,4)])

{(1, 2), (3, 4)}